# A ML approach to a FICO problem

Fico is the industry standard for determining credit worthiness. However, research shows that FICO is overused. FICO's usage fits the idiom "square peg in a round hole".

Evidence:

* A [research study](https://www.stlouisfed.org/publications/regional-economist/october-2008/did-credit-scores-predict-the-subprime-crisis) by the Federal Reserve Bank of St. Louis demonstrated the failure of FICO socres to determine credit worthiness during the subprime mortgage crisis.
> Given the nature of FICO scores, one might expect to find a relationship between borrowers’ scores and the incidence of default and foreclosure.....FICO scores have not indicated that relationship....higher FICO scores have been associated with bigger increases in default rates over time.
* [Studies prove](https://www.marketwatch.com/story/your-digital-footprint-could-provide-a-more-accurate-credit-score-2018-05-03) that digital footprints outperform FICO.
> Those who order from mobile phones are three times as likely to default as those who order from desktops. A customer who uses her name in her email address is 30 percent less likely to default than one who doesn’t. Those who shop between noon and 6 p.m. are half as likely to default as midnight to 6 a.m. buyers
* FICO depends on credit usage (CU). CU is unreliable because it fluctuates through economic cycles. [Credit usage amongs millenials dropped](http://fortune.com/2018/02/27/why-millennials-are-ditching-credit-cards/), especially during the 2008 recession. But being debt conscious is financially desirable.
* Paying timely minimums on credit statements results in a strong score. However, this results in mounting debt.
* Credit utilization ratios factor heavily into FICO ( $util = \frac{debt}{availCredt}$ ). A healthy utilization is < 0.3. Credit card companies encourage opening several credit lines to improve this ratio.